In [2]:
# import libraries
import os
import pathlib
import pickle
import uuid
import mlflow
import pandas as pd
import numpy as np
from zipfile import ZipFile

from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from prefect import flow, task
from prefect.artifacts import create_markdown_artifact
from datetime import date


In [3]:
dataset = "stack-overflow"
year = 2023

# input_file = f'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/{dataset}-developer-survey-{year:04d}.zip'
input_file = f'../../data/raw/{dataset}-developer-survey-{year:04d}.zip'
output_file = f'output/score-{dataset}-{year:04d}.csv'


In [6]:
# load saved model from gcs bucket
RUN_ID = "8ac5e4553c464697a9d70d833458e3d2"
# RUN_ID = os.getenv('RUN_ID')
logged_model = f"gs://mlflow-cb-stack-overflow/1/{RUN_ID}/artifacts/models_mlflow"

# Load model as a PyFuncModel.
model = mlflow.pyfunc.load_model(logged_model)

In [7]:
mlflow.pyfunc.get_model_dependencies(logged_model)

2023/08/19 23:24:37 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmp92t_9v05/models_mlflow/requirements.txt'.


'/tmp/tmp92t_9v05/models_mlflow/requirements.txt'

In [ ]:
%pip install -r '/tmp/tmp9slo8sbj/models_mlflow/requirements.txt'

In [9]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.catboost
  run_id: 8ac5e4553c464697a9d70d833458e3d2

In [10]:
def load_data(filename: str) -> pd.DataFrame:
    print("Loading data from the zip file...")

    # Load data from the zip file
    with ZipFile(filename) as zip_file:
        df = pd.read_csv(zip_file.open("survey_results_public.csv"))

    print("Data loaded successfully.")
    return df

def preprocess_data(df: pd.DataFrame):
    print("Preprocessing data...")

    # Drop 'ResponseId' column and remove duplicates
    df = df.drop(["ResponseId"], axis=1).drop_duplicates()

    # Set the target variable
    target = "ConvertedCompYearly"
    # Convert compensations into kUSD/year
    df[target] = df[target] * 1e-3

    print("Filtering outliers based on the target label...")
    # Filter outliers by selecting records with target label more than 1k USD/year
    df = df[df[target] > 1.0]

    # Further exclude 2% of smallest and 2% of highest salaries
    P = np.percentile(df[target], [2, 98])
    df = df[(df[target] > P[0]) & (df[target] < P[1])]

    print("Converting YearsCode, YearsCodePro, and WorkExp to integers...")

    # Convert YearsCode, YearsCodePro, and WorkExp to integers
    def clean_years(x):
        if x == "Less than 1 year":
            return 0
        elif x == "More than 50 years":
            return 51
        else:
            return x

    df["YearsCode"] = df["YearsCode"].apply(clean_years).fillna(-1).astype(int)
    df["YearsCodePro"] = df["YearsCodePro"].apply(clean_years).fillna(-1).astype(int)
    df["WorkExp"] = df["WorkExp"].fillna(-1).astype(int)

    # Fill NaN values with 'none'
    df = df.fillna("none")

    print("Dropping unused columns...")
    # Drop unused columns
    cols2drop = [
        "Q120",
        "MainBranch",
        "CodingActivities",
        "Knowledge_1",
        "Knowledge_2",
        "Knowledge_3",
        "Knowledge_4",
        "Knowledge_5",
        "Knowledge_6",
        "Knowledge_7",
        "Knowledge_8",
        "Frequency_1",
        "Frequency_2",
        "Frequency_3",
        "PurchaseInfluence",
        "TechList",
        "BuyNewTool",
        "Currency",
        "CompTotal",
        "LanguageWantToWorkWith",
        "DatabaseWantToWorkWith",
        "PlatformWantToWorkWith",
        "WebframeWantToWorkWith",
        "MiscTechWantToWorkWith",
        "ToolsTechWantToWorkWith",
        "NEWCollabToolsWantToWorkWith",
        "OpSysPersonal use",
        "OfficeStackAsyncWantToWorkWith",
        "OfficeStackSyncWantToWorkWith",
        "AISearchWantToWorkWith",
        "AIDevWantToWorkWith",
        "NEWSOSites",
        "SOVisitFreq",
        "SOAccount",
        "SOPartFreq",
        "SOComm",
        "SOAI",
        "AISelect",
        "AISent",
        "AIAcc",
        "AIBen",
        "AIToolInterested in Using",
        "AIToolCurrently Using",
        "AIToolNot interested in Using",
        "AINextVery different",
        "AINextNeither different nor similar",
        "AINextSomewhat similar",
        "AINextVery similar",
        "AINextSomewhat different",
        "SurveyLength",
        "SurveyEase",
        "TimeSearching",
        "TimeAnswering",
    ]
    
 
    df = df.drop(cols2drop, axis=1)

    print("Data preprocessing completed successfully.")
    return df

In [11]:
df = load_data(input_file)
df = preprocess_data(df)
y_pred = model.predict(df)

Loading data from the zip file...
Data loaded successfully.
Preprocessing data...
Filtering outliers based on the target label...
Converting YearsCode, YearsCodePro, and WorkExp to integers...
Dropping unused columns...
Data preprocessing completed successfully.


In [12]:
y_pred

array([196.54656578, 210.36691124, 151.18018641, ...,   7.95447818,
        60.5302622 ,  35.00976909])

In [13]:
df.shape

(45430, 30)

In [14]:
df.head()

,Age,Employment,RemoteWork,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,DevType,...,OfficeStackAsyncHaveWorkedWith,OfficeStackSyncHaveWorkedWith,AISearchHaveWorkedWith,AIDevHaveWorkedWith,TBranch,ICorPM,WorkExp,ProfessionalTech,Industry,ConvertedCompYearly
1,25-34 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,Other,18,9,"Senior Executive (C-Suite, VP, etc.)",...,Asana;Basecamp;GitHub Discussions;Jira;Linear;...,Cisco Webex Teams;Discord;Google Chat;Google M...,ChatGPT,GitHub Copilot,Yes,People manager,10,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",285.000
2,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,none,27,23,"Developer, back-end",...,Markdown File;Stack Overflow for Teams,Microsoft Teams;Slack;Zoom,none,none,Yes,Individual contributor,23,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",250.000
3,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,none,12,7,"Developer, front-end",...,Jira,Discord;Google Meet;Microsoft Teams;Slack;Zoom,none,none,Yes,Individual contributor,7,Automated testing;Continuous integration (CI) ...,none,156.000
4,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,Other;Codecademy;edX,6,4,"Developer, full-stack",...,Confluence;Jira;Notion,Discord;Google Meet;Slack;Zoom,ChatGPT,none,Yes,Individual contributor,6,Microservices;Automated testing;Observability ...,Other,23.456
5,35-44 years old,"Employed, full-time",Remote,Some college/university study without earning ...,Books / Physical media;Colleague;Online Course...,Formal documentation provided by the owner of ...,Other,21,21,"Developer, back-end",...,Jira;Markdown File;Notion;Stack Overflow for T...,Google Meet;Microsoft Teams;Slack;Zoom,ChatGPT;Google Bard AI;Neeva AI,GitHub Copilot;Tabnine,Yes,Individual contributor,22,DevOps function;Microservices;Observability to...,Other,96.828


In [15]:
df.to_csv('output/df_result_temp.csv', index=False)

In [16]:
df_result = pd.read_csv('output/df_result_temp.csv')


In [17]:
df_result.head()

,Age,Employment,RemoteWork,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,DevType,...,OfficeStackAsyncHaveWorkedWith,OfficeStackSyncHaveWorkedWith,AISearchHaveWorkedWith,AIDevHaveWorkedWith,TBranch,ICorPM,WorkExp,ProfessionalTech,Industry,ConvertedCompYearly
0,25-34 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,Other,18,9,"Senior Executive (C-Suite, VP, etc.)",...,Asana;Basecamp;GitHub Discussions;Jira;Linear;...,Cisco Webex Teams;Discord;Google Chat;Google M...,ChatGPT,GitHub Copilot,Yes,People manager,10,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",285.000
1,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,none,27,23,"Developer, back-end",...,Markdown File;Stack Overflow for Teams,Microsoft Teams;Slack;Zoom,none,none,Yes,Individual contributor,23,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",250.000
2,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,none,12,7,"Developer, front-end",...,Jira,Discord;Google Meet;Microsoft Teams;Slack;Zoom,none,none,Yes,Individual contributor,7,Automated testing;Continuous integration (CI) ...,none,156.000
3,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,Other;Codecademy;edX,6,4,"Developer, full-stack",...,Confluence;Jira;Notion,Discord;Google Meet;Slack;Zoom,ChatGPT,none,Yes,Individual contributor,6,Microservices;Automated testing;Observability ...,Other,23.456
4,35-44 years old,"Employed, full-time",Remote,Some college/university study without earning ...,Books / Physical media;Colleague;Online Course...,Formal documentation provided by the owner of ...,Other,21,21,"Developer, back-end",...,Jira;Markdown File;Notion;Stack Overflow for T...,Google Meet;Microsoft Teams;Slack;Zoom,ChatGPT;Google Bard AI;Neeva AI,GitHub Copilot;Tabnine,Yes,Individual contributor,22,DevOps function;Microservices;Observability to...,Other,96.828


In [18]:
df_result['predicted_ConvertedCompYearly'] = y_pred
df_result['diff'] = df_result['ConvertedCompYearly'] - df_result['predicted_ConvertedCompYearly']

In [19]:
df_result.head()

,Age,Employment,RemoteWork,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,DevType,...,AISearchHaveWorkedWith,AIDevHaveWorkedWith,TBranch,ICorPM,WorkExp,ProfessionalTech,Industry,ConvertedCompYearly,predicted_ConvertedCompYearly,diff
0,25-34 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,Other,18,9,"Senior Executive (C-Suite, VP, etc.)",...,ChatGPT,GitHub Copilot,Yes,People manager,10,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",285.000,196.546566,88.453434
1,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,none,27,23,"Developer, back-end",...,none,none,Yes,Individual contributor,23,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",250.000,210.366911,39.633089
2,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,none,12,7,"Developer, front-end",...,none,none,Yes,Individual contributor,7,Automated testing;Continuous integration (CI) ...,none,156.000,151.180186,4.819814
3,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,Other;Codecademy;edX,6,4,"Developer, full-stack",...,ChatGPT,none,Yes,Individual contributor,6,Microservices;Automated testing;Observability ...,Other,23.456,33.294622,-9.838622
4,35-44 years old,"Employed, full-time",Remote,Some college/university study without earning ...,Books / Physical media;Colleague;Online Course...,Formal documentation provided by the owner of ...,Other,21,21,"Developer, back-end",...,ChatGPT;Google Bard AI;Neeva AI,GitHub Copilot;Tabnine,Yes,Individual contributor,22,DevOps function;Microservices;Observability to...,Other,96.828,123.139417,-26.311417


In [20]:
df_result.sample(5)

,Age,Employment,RemoteWork,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,DevType,...,AISearchHaveWorkedWith,AIDevHaveWorkedWith,TBranch,ICorPM,WorkExp,ProfessionalTech,Industry,ConvertedCompYearly,predicted_ConvertedCompYearly,diff
14310,35-44 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Blogs with tips and tricks;Books;How-to videos...,Udemy,12,12,"Developer, full-stack",...,none,none,Yes,Individual contributor,12,DevOps function;Microservices;Continuous integ...,Healthcare,110.000,109.000000,1.000000
23825,25-34 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","School (i.e., University, College, etc)",none,none,10,6,"Developer, full-stack",...,none,none,Yes,Individual contributor,6,DevOps function;Microservices;Continuous integ...,Other,200.000,150.757697,49.242303
23142,55-64 years old,"Employed, full-time",Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","School (i.e., University, College, etc)",none,none,25,25,"Developer, back-end",...,none,none,No,none,-1,none,none,140.000,164.378007,-24.378007
12418,35-44 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;Hackathons (v...,Formal documentation provided by the owner of ...,Udemy;Coursera,21,17,Engineering manager,...,Bing AI;ChatGPT;WolframAlpha,GitHub Copilot;Tabnine,No,none,-1,none,none,149.925,138.733876,11.191124
16417,55-64 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,Other;Codecademy,43,34,"Developer, back-end",...,none,none,Yes,Individual contributor,34,None of these,"Information Services, IT, Software Development...",105.000,163.183530,-58.183530


In [21]:
df_result.to_csv(output_file, index=False)